# Demo 02: View water column data

## Introduction

This Jupyter Notebook contains an introduction to viewing water column data using Ping.

## Here you will learn

- 

## Short version

At the end of the notebook you will find the important parts of this notebook, compiled into a single cell


## ---

### First: Import the necessary modules

In [ ]:
# use widgets as matplotlib backend
%matplotlib widget

# imports
from matplotlib import pyplot as plt
import os
import sys
import numpy as np
from collections import defaultdict
from time import time
from tqdm.auto import tqdm #progress bar, auto selects a backend based on the environment e.g. for a jupyter notebook it will use the widget version

from typing import Union, Tuple

import matplotlib as mpl
import matplotlib.dates as mdates

import datetime as dt

# import the file object for opening kongsberg files
# Note: function and library naming to be discussed
from themachinethatgoesping.echosounders import kongsbergall          # This is the filereader module for .all/.wcd files
from themachinethatgoesping.echosounders import pingtools          # This is the filereader module for .all/.wcd files
from themachinethatgoesping.echosounders import index_functions # This is the module for creating and using indexes to speed up file initialization
import themachinethatgoesping.tools as ptools                   # Some tools for working with ping (here we use timeconv for timestamp conversion)
import themachinethatgoesping.algorithms as palgorithms                   # Some tools for working with ping (here we use timeconv for timestamp conversion)
import themachinethatgoesping.navigation as pnav                   # Some tools for working with ping (here we use timeconv for timestamp conversion)

import themachinethatgoesping.pingprocessing.watercolumn.make_image as mi
import themachinethatgoesping.pingprocessing.watercolumn.helper.make_image_helper as mi_hlp

from themachinethatgoesping.pingprocessing import filter_pings, overview, split_pings, group_pings
from themachinethatgoesping.pingprocessing import watercolumn
gp = palgorithms.geoprocessing
import pandas as pd

# simplify creating figures
mpl.rcParams['figure.dpi'] = 100
close_plots: bool = True

def create_figure(name: str, return_ax: bool = True) -> Union[Tuple[plt.Figure, plt.Axes], plt.Figure]:
    """Helper function to create a figure with a given name that returns the figure and axis
    """
    if close_plots:
        plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name

    if return_ax:
        return fig, fig.subplots()
    return fig

In [ ]:
from collections import OrderedDict

class fake_tqdm(object):
    def __init__(self, w_prg):
        self.w_prg = w_prg
        
    def __call__(self, list_like):
        self.list_like = list_like
        self.list_iter = iter(list_like)
        self.index = 0
        self.total = len(list_like)
        self.w_prg.max = self.total
        self.w_prg.value = 0
        return self
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.index += 1
        self.w_prg.value = self.index
        return next(self.list_iter)
    
    def __len__(self):
        return self.total
    
    def update(self):
        self.index += 1
        next(self.list_iter)
        self.w_prg.value = self.index
        
    def close(self):
        pass

### Find files to open

This is a simple python loop to find .all and .wcd files in the specified directories

In [ ]:
#%%timeit -n1 -r1

# Define the folder(s) to search for Kongsberg files
# Note: subdirectories will be searched as well
folders = []
folders.append("/home/data/turbeams/TURBEAMS 2023-12/")

# Initialize lists for storing file paths and names
files = index_functions.find_files(folders, [".all",'wcd'])
files.sort()

## Open files


In [ ]:
def update_navigation_cache(fm, cached_navigation):
    iterator = tqdm(fm.navigation_interface.per_file())
    configuration_interface_list = fm.configuration_interface.per_file()

    for interface, config_per_file in zip(iterator, configuration_interface_list):
        f = interface.get_file_path()
        if not f in cached_navigation.keys():
            iterator.set_postfix_str(f'update navigation for {f}')

            if not config_per_file.is_initialized():
                config_per_file.init_from_file()
    
            cached_navigation[f] = interface.read_navigation_data()
    
    index_functions.update_navigation_cache_files(cached_navigation)

In [ ]:
#%%timeit -n1 -r1

cached_index = index_functions.load_index_cache_files(file_paths=files)
cached_navigation = index_functions.load_navigation_cache_files(file_paths=files)
#cached_index = {}
#cached_navigation = {}
fm = kongsbergall.KongsbergAllFileHandler_mapped(files[:10], cached_index=cached_index, init = False)

index_functions.update_index_cache_files(fm.get_cached_file_index())
update_navigation_cache(fm, cached_navigation)

# initialize interfaces
fm.navigation_interface.init_from_file_or_cache(cached_navigation)
fm.init_interfaces()

In [ ]:
valid_times = [
    # (dt.datetime(2023, 3, 18, 16, 00, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 18, 18, 30, tzinfo=dt.timezone.utc)), # PAC1
    # (dt.datetime(2023, 3, 19, 5, 25, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 19, 7, 30, tzinfo=dt.timezone.utc)), # PAC2
    # (dt.datetime(2023, 3, 19, 17, 15, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 19, 18, 37, tzinfo=dt.timezone.utc)), # PAC3
]

if valid_times:
    pings = []
    for ping in tqdm(fm.pings()):
        ping_dt = ping.get_datetime()
        for pt in valid_times:
            if ping_dt >= pt[0] and ping_dt <= pt[1]:
                pings.append(ping)
else:
    pings = fm.pings()

print(f"Found {len(pings)}/{len(fm.pings())} pings in valid time range")

In [ ]:
pings = filter_pings.by_features(pings,['watercolumn.amplitudes'],True)
pings = filter_pings.by_features(pings,['bottom'],True)
#pings = split_pings.by_channel_id(pings)['TRX-221']

times = [ps[0].get_timestamp() for ps in split_pings.by_time_difference(pings, 30, True).values()]


ping_containers = split_pings.by_time_blocks(pings, times)

ping_containers = {ptools.timeconv.unixtime_to_datestring(k, format='%d-%m-%Y %H:%M:%S') : v for k,v in ping_containers.items()}

stats = overview.get_ping_overview(ping_containers,True)

for k,pings in ping_containers.items():
    print(k,len(pings))

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import CRS

def to_gpd(latitude,longitude):
    newdata = gpd.GeoDataFrame(geometry=gpd.GeoSeries())
    coordinates = list(zip(longitude, latitude))
    line = LineString(coordinates)
    newdata.loc[0, 'geometry'] = line
    newdata.crs = CRS.from_epsg(4326)
    
    return newdata
    

In [ ]:
interesting_times = []
# interesting_times =[
#     ("01 PAC1 VPR", dt.datetime(2023, 3, 18, 16, 6, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 18, 17, 0, tzinfo=dt.timezone.utc), {'color':'Yellow','marker':'o'}),
#     ("02 PAC1 CTD 1", dt.datetime(2023, 3, 18, 17, 13, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 18, 17, 21, tzinfo=dt.timezone.utc), {'color':'Red','marker':'o'}),
#     ("02 PAC1 CTD 2", dt.datetime(2023, 3, 18, 18, 0, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 18, 18, 25, tzinfo=dt.timezone.utc), {'color':'Red','marker':'o'}),
#     ("03 PAC3 VPR", dt.datetime(2023, 3, 19, 17, 15, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 19, 17, 52, tzinfo=dt.timezone.utc), {'color':'Yellow','marker':'+'}),
#     ("04 PAC3 CTD ", dt.datetime(2023, 3, 19, 18, 11, tzinfo=dt.timezone.utc), dt.datetime(2023, 3, 19, 18, 25, tzinfo=dt.timezone.utc), {'color':'Red','marker':'+'}),
# ]

In [ ]:
import rasterio.plot as rioplt
import rasterio as rio

background_map = rio.open('qgis/BPNS_latlon.tiff')

fig,ax = overview.nav_plot.create_figure('Navigation data', aspect='auto')
rioplt.show(background_map,ax=ax,cmap='Greys_r')

all_stats = defaultdict(list)

for k,pings in ping_containers.items():
    #if k != '18-03-2023 16:05:23':
    #if k != '19-03-2023 05:25:23':
    #if k != "19-03-2023 17:15:00":
    #    continue

    if len(pings) < 10:
        continue
        
    pings = list(split_pings.by_channel_id(pings).values())[0]
    stats = overview.get_ping_overview(pings)
    overview.nav_plot.plot_latlon(stats.variables['latitude'], stats.variables['longitude'], ax, survey_name=k)

    all_stats['latitude'].extend(stats.variables['latitude'])
    all_stats['longitude'].extend(stats.variables['longitude'])
    all_stats['timestamp'].extend(stats.variables['timestamp'])
    
    !mkdir -p shapefiles
    outfile = 'shapefiles/' + k+'.shp'
    gpdate = to_gpd(stats.variables['latitude'],stats.variables['longitude'])
    gpdate.to_file(outfile)
    print(k, len(pings))

dataframe = pd.DataFrame(all_stats)

for name, time_start, time_end, kwargs in interesting_times:
    data = dataframe.loc[(dataframe['timestamp'] >= time_start.timestamp()) & (dataframe['timestamp'] <= time_end.timestamp())]
    if len(data) > 0:
        ax.plot(data['longitude'],data['latitude'],label=name, **kwargs)

fig.legend()
#ax.set_xlim(2.537, 2.6)
#ax.set_ylim(51.6, 51.66)

In [ ]:
if False:
    !mkdir -p plots
    fig.savefig('plots/overview.png')
    fig.savefig('plots/overview.svg')

In [ ]:

def get_time(ping_group):
    times=[]
    for ping in ping_group.values():
        times.append(ping.get_timestamp())
    return np.min(times)
    
def get_datetime(ping_group):
    times=[]
    for ping in ping_group.values():
        times.append(ping.get_datetime())
    return np.min(times)


def get_latlon(ping_group):
    lats = []
    lons = []
    for ping in ping_group.values():
        geo = ping.get_geolocation()
        lats.append(geo.latitude)
        lons.append(geo.longitude)
    return np.mean(lats),np.mean(lons)

def get_echosounder_depth(ping_group):
    depth = []
    for ping in ping_group.values():
        depth.append(ping.get_geolocation().z)
    return np.nanmax(depth)
    
def get_bottom_depth(ping_group, pss = None):
    depth = []
    for ping in ping_group.values():
        try:
            if ping.has_bottom():
                if pss is not None:
                    sel = pss.apply_selection(ping.bottom)
                    depth.append(np.nanmin(ping.bottom.get_xyz(sel).z) + ping.get_geolocation().z)
                else:
                    depth.append(np.nanmin(ping.bottom.get_xyz().z) + ping.get_geolocation().z)
        except Exception as e:
            print(e)
            pass

    if len(depth) > 0:
        return np.nanmin(depth)

    return None
        

def get_resolution(ping_group):
    sound_velocities = []
    sample_interval = np.nan
    for ping in ping_group.values():        
        sound_velocities.append(ping.watercolumn.get_sound_speed_at_transducer())
        
        if np.isnan(sample_interval):
            sample_interval = ping.watercolumn.get_sample_interval()
        elif sample_interval != ping.watercolumn.get_sample_interval():
            raise RuntimeError(f"Ping group sample intervals do not match! [{sample_interval} != {ping.watercolumn.get_sample_interval()}]")
        
    return np.nanmean(sound_velocities) * sample_interval * 0.5

In [ ]:
I = sorted(list(ping_containers.keys()))
# I = ['13-12-2023 11:28:08'] # transect MOW1
#I = ['13-12-2023 21:29:08'] # transect Transect1
#I = ['14-12-2023 16:03:59'] # W05
# I = ['14-12-2023 22:36:32'] # Transect2
#I = ['15-12-2023 19:12:30', '15-12-2023 20:58:18'] # W08
#I = ['16-12-2023 12:30:18', '17-12-2023 00:55:56'] # OutsideArea

# I = [
#     '14-12-2017 09:27:00',
# ]

timestrings=[]
ping_groups=[]
for i in tqdm(I):
    pings = [p for p in ping_containers[i]]
    pings_group = group_pings.dual_head(pings, True)
    #pings_group = dual_head(pings, True)
    pnr1 = len(ping_groups)
    ping_groups.extend(pings_group)
    pnr2 = len(ping_groups)-1
    
    dt1 = get_datetime(pings_group[0])
    dt2 = get_datetime(pings_group[-1])
    timestrings.append((pnr1,ptools.timeconv.datetime_to_datestring(dt1,format='%d-%m-%Y %H:%M:%S'),dt1,mdates.date2num(dt1)))
    timestrings.append((pnr2,ptools.timeconv.datetime_to_datestring(dt2,format='%d-%m-%Y %H:%M:%S'),dt2,mdates.date2num(dt2)))
    
for pg in tqdm(ping_groups):
    for trid,ping in pg.items():
        #ping.load()
        pass

In [ ]:
from typing import List, Tuple
import numpy as np
def get_filtered_pings_by_time_step(pings: List, 
                          min_num_pings_per_group: int = 2,
                          split_group_size: int = 10000,
                          min_resolution: float = 0.01, 
                          max_resolution_change_per_group: float = 1.1,
                          max_pingrate_change_per_group: float = 1.01,
                          max_pingdistance_change_per_group: float = 1.1,
                          min_pingdistance_to_consider = 1) -> List[Tuple]:
    """
    Filters and groups pings based on time steps and other criteria.

    Args:
        pings (list): List of pings to be filtered and grouped.
        max_pings (int, optional): Maximum number of pings to consider. Defaults to 1000000000000000000000.
        min_num_pings_per_group (int, optional): Minimum number of pings required in each group. Defaults to 1.
        split_group_size (int, optional): Maximum number of pings allowed in each group. Defaults to 10000.
        min_resolution (float, optional): Minimum resolution for pings. Defaults to 0.01.
        max_resolution_change_per_group (float, optional): Maximum allowed resolution change per group. Defaults to 1.1.

    Returns:
        list: List of tuples containing filtered and grouped pings. Each tuple contains:
            - ping numbers
            - median resolution
            - corresponding ping groups
    """
    times_times: List[List[float]] = []  # List to store ping times for each group
    times_pnr: List[List[int]] = []  # List to store ping numbers for each group
    times_resolutions: List[List[float]] = []  # List to store resolutions for each group
    times_latlon = []
    old_time: float = 0  # Variable to store previous ping time
    A: int = 0

    current_min_resolution: float = np.nan
    current_max_resolution: float = np.nan  
    current_min_pingrate: float = np.nan  
    current_max_pingrate: float = np.nan  
    current_min_pingdistance: float = np.nan  
    current_max_pingdistance: float = np.nan  
    
    # Sort ping times (ping numbers) in array, break when difference between pings > split_time_diff
    for i, ping in enumerate(tqdm(pings, delay=3)):
        ping_time: float = get_time(ping)
        ping_latlon = get_latlon(ping)
        
        # Compute resolution        
        resolution: float = np.max((get_resolution(ping), min_resolution))
        pingrate = 0
        pingdistance = 0
        
        if times_times:
            if times_times[-1]:
                if ping_time <= times_times[-1][-1]:
                    print("WARNING: ping times are not sorted by time! Ignoring pings")
                    continue

                pingrate = ping_time - times_times[-1][-1]
                pingdistance = pnav.navtools.compute_latlon_distance_m(ping_latlon, times_latlon[-1][-1])
                
        
        # Split if:
        #   - ping time > split_time_diff
        #   - times_times is empty
        #   - there are more pings in the group than specified by split_group_size
        #   - the resolution change is too large
        if not times_times or len(times_pnr[-1]) >= split_group_size:
            times_times.append([])
            times_pnr.append([])     
            times_resolutions.append([])   
            times_latlon.append([])
            current_min_resolution = resolution
            current_max_resolution = resolution
            current_min_pingrate: float = pingrate 
            current_max_pingrate: float = pingrate  
            current_min_pingdistance: float = pingdistance  
            current_max_pingdistance: float = pingdistance  
            
        elif resolution / current_min_resolution > max_resolution_change_per_group or current_max_resolution / resolution > max_resolution_change_per_group:
            times_times.append([])
            times_pnr.append([])     
            times_resolutions.append([])   
            times_latlon.append([])
            current_min_resolution = resolution
            current_max_resolution = resolution
            current_min_pingrate: float = pingrate 
            current_max_pingrate: float = pingrate  
            current_min_pingdistance: float = pingdistance  
            current_max_pingdistance: float = pingdistance  

        elif len(times_times) > 1:
            if pingrate / current_min_pingrate > max_pingrate_change_per_group or current_max_pingrate / pingrate > max_pingrate_change_per_group:
                print(f'split time 1 {pingrate} {current_min_pingrate} {current_max_pingrate}')
                print(f'split time 2 {pingrate / current_min_pingrate} {current_max_pingrate / pingrate} {max_pingrate_change_per_group}')
                times_times.append([])
                times_pnr.append([])     
                times_resolutions.append([])   
                times_latlon.append([])
                current_min_resolution = resolution
                current_max_resolution = resolution
                current_min_pingrate: float = pingrate 
                current_max_pingrate: float = pingrate  
                current_min_pingdistance: float = pingdistance  
                current_max_pingdistance: float = pingdistance  
            elif pingdistance > min_pingdistance_to_consider and (pingdistance / current_min_pingdistance > max_pingdistance_change_per_group or current_max_pingdistance / pingdistance > max_pingdistance_change_per_group):
                print(f'split distance 1 {pingdistance} {current_min_pingdistance} {current_max_pingdistance}')
                print(f'split distance 2 {pingdistance / current_min_pingdistance} {current_max_pingdistance / pingdistance} {max_pingdistance_change_per_group}')
                times_times.append([])
                times_pnr.append([])     
                times_resolutions.append([])   
                times_latlon.append([])
                current_min_resolution = resolution
                current_max_resolution = resolution
                current_min_pingrate: float = pingrate 
                current_max_pingrate: float = pingrate  
                current_min_pingdistance: float = pingdistance  
                current_max_pingdistance: float = pingdistance  
            
        current_min_resolution = np.nanmin((resolution, current_min_resolution))
        current_max_resolution = np.nanmax((resolution, current_max_resolution))
        current_min_pingrate = np.nanmin((pingrate, current_min_pingrate))
        current_max_pingrate = np.nanmax((pingrate, current_max_pingrate))
        current_min_pingdistance = np.nanmin((pingdistance, current_min_pingdistance))
        current_max_pingdistance = np.nanmax((pingdistance, current_max_pingdistance))
            
        times_resolutions[-1].append(resolution)
        times_times[-1].append(ping_time)
        times_pnr[-1].append(i)
        times_latlon[-1].append(ping_latlon)
        old_time = ping_time
    
    # Compute number of samples per section
    times_num_pings: np.ndarray = np.array([len(t) for t in times_times])
        
    # Compute ping times
    times_pingtimes = [np.linspace(tt[0], tt[-1], tn) for tt, tn in zip(times_times, times_num_pings)]
    
    # Interpolate ping numbers for the ping times
    times_pings: List[Tuple] = []
    pga: np.ndarray = np.array(pings)
    for tt, tp, tpt, tres in zip(times_times, times_pnr, times_pingtimes, times_resolutions):
        try:                
            tp_interpolator = ptools.vectorinterpolators.NearestInterpolator(tt, tp, "fail")
            pingnumbers = np.unique(np.array(tp_interpolator(tpt)).astype(int))

            if len(pingnumbers) >= min_num_pings_per_group:
                times_pings.append((
                    pingnumbers,
                    np.nanmedian(tres),
                    pga[pingnumbers])
                )
        except Exception as e:
            fig, ax = create_figure('exception')
            ax.plot(tt, tp, marker='+')
            raise e
    
    return times_pings
        
        
# filtered_pings = get_filtered_pings_by_time_step(ping_groups,max_pings=100000)

# for n,r,p in filtered_pings:
#     print(r,len(n))
# np.sum([len(np[-1]) for np in filtered_pings])

In [ ]:
def get_time_error(echogram_times, ping_time):
    if not echogram_times:
        return np.nan

    echogram_times = echogram_times.copy()
    echogram_times.append(ping_time)    
    echogram_times = np.array(echogram_times)

    global linpace_times,tdiff,ping_diff

    linpace_times = np.linspace(echogram_times[0], echogram_times[-1], len(echogram_times))
    tdiff = linpace_times[1] - linpace_times[0]
    
    time_error = np.abs(echogram_times-linpace_times)

    return time_error/tdiff


In [ ]:
class EchogramGroup():
    def __init__(self):
        self.pings = []
        self.pnr = []
        self.resolution = None

    def add(self, ping_group, pnr, resolution):
        self.pings.append(ping_group)
        self.pnr.append(pnr)
        self.resolution = resolution

    def to_tuple(self):
        return self.pnr, self.resolution, self.pings

    def __len__(self):
        return len(self.pings)

In [ ]:
from typing import List, Tuple
import numpy as np

def create_echogram_groups(
    pings: List, 
    min_num_pings_per_group: int = 2,
    max_section_size: int = 10000,
    max_time_diff_error = 1) -> List[Tuple]:

    echogram_groups = [EchogramGroup()]
    echogram_times = []

    for pnr,pg in enumerate(tqdm(pings)):
        ping_time = get_time(pg)
        
        if np.max(get_time_error(echogram_times, ping_time)) > max_time_diff_error:
            echogram_times = []
            echogram_groups.append(EchogramGroup())

        echogram_times.append(ping_time)
        echogram_groups[-1].add(pg, pnr, get_resolution(pg))

    return [eg.to_tuple() for eg in echogram_groups]
    
# echogram_groups = create_echogram_groups(ping_groups,max_time_diff_error=1)

# for i,eg in enumerate(echogram_groups):
#     print(i,len(eg[0]))

In [ ]:
def unixtime_to_datetime(unixtime):
    # if unixtime is a container convert each element
    if hasattr(unixtime, '__iter__'):
        return [dt.datetime.fromtimestamp(t, dt.timezone.utc) for t in unixtime]

    # otherwise convert the single value
    return dt.datetime.fromtimestamp(unixtime, dt.timezone.utc)

def datetime_to_unixtime(datetime):
    # if datetime is a container convert each element
    if hasattr(datetime, '__iter__'):
        return [t.timestamp() for t in datetime]

    # otherwise convert the single value
    return datetime.timestamp()

def unixtime_to_mdate(unixtime):
    return mdates.date2num(unixtime_to_datetime(unixtime))

def mdate_to_unixtime(mdate):
    return datetime_to_unixtime(mdates.num2date(mdate))

unixtime_to_datetime(1631635200.0)
datetime_to_unixtime(dt.datetime(2021,9,14,0,0,0,0,dt.timezone.utc))

times = unixtime_to_datetime([1631635200.0,1631635230])
datetime_to_unixtime(times)

times = unixtime_to_mdate([1631635200.0,1631635230])
mdate_to_unixtime(times)

In [ ]:
#set the beam angle range in degrees
beam_angle_min = -3.5
beam_angle_max = 3.5
beam_step = 1
sample_step = 1
#max_samples = 2500
linear_mean = True

min_resolution=0.000000001
max_resolution_change_per_group=1.11
max_pingrate_change_per_group=1.11
max_pingdistance_change_per_group=5

max_pings = 1000000
split_group_size = 10000

ping_step=1
if len(ping_groups) > max_pings:
    ping_step = np.round(len(ping_groups) / max_pings).astype(int)
print(ping_step)

#bss = pingtools.BeamSampleSelection(sample_step_ensemble=sample_step)
pss = pingtools.PingSampleSelector()
pss.select_beam_range_by_angles(beam_angle_min,beam_angle_max, beam_step)
#pss.select_sample_range_by_numbers(0, max_samples, sample_step)
pss.set_sample_step(sample_step)


# pn_fp = get_filtered_pings_by_time_step(
#     pings = ping_groups[::ping_step],
#     split_group_size=split_group_size,
#     min_resolution=min_resolution,
#     max_resolution_change_per_group=max_resolution_change_per_group,
#     max_pingrate_change_per_group=max_pingrate_change_per_group,
#     max_pingdistance_change_per_group=max_pingdistance_change_per_group)

pn_fp = create_echogram_groups(ping_groups,max_time_diff_error=0.4999)
    
prg = tqdm(total=sum([len(f[-1]) for f in pn_fp]))
print('number of echogram groups:',len(pn_fp))


echograms = []
    
last_pos = None
last_distance = 0
for ping_numbers,z_resolution,filtered_pings in pn_fp:
    
    num_s  = []
    sound_velocities = []
    sample_intervals = []
    echosounder_depths = []
    echosounder_depths_times = []
    bottom_depths = []
    bottom_depths_times = []
    frequencies = []
    for i,pg in enumerate(filtered_pings):
        cf = set()
        for p in pg.values():
            try:
                sel = pss.apply_selection(p.watercolumn)
                sample_offset = int(round(p.get_geolocation().z / z_resolution))
                num_s.append(np.max(p.watercolumn.get_number_of_samples_per_beam(sel)) + sample_offset)

                               
                sound_velocities.append(p.watercolumn.get_sound_speed_at_transducer())
                sample_intervals.append(p.watercolumn.get_sample_interval())
                p.watercolumn.get_tx_signal_parameters()

                freq_per_sector = [signal.center_frequency for signal in p.watercolumn.get_tx_signal_parameters()]
                sector_numbers = p.watercolumn.get_tx_sector_per_beam()[sel.get_beam_numbers()]
                cf.update({freq_per_sector[sector_n] for sector_n in sector_numbers})
                
            except IndexError as e:
                print("error:",i,e,"|",type(e))
            except ValueError as e:
                print("error:",i,e,"|",type(e))
            except RuntimeError as e:
                print("error:",i,e,"|",type(e))

        
        echosounder_depths.append(get_echosounder_depth(pg))
        echosounder_depths_times.append(p.get_timestamp())
        frequencies.append(tuple(cf))

        bd = get_bottom_depth(pg,pss)
        if bd is not None:
            bottom_depths.append(bd)
            bottom_depths_times.append(get_time(pg))
                
    num_s = np.array(num_s)
    max_s = np.max(num_s[num_s < np.quantile(num_s,0.95)*1.3])
    max_depth = np.median(sound_velocities) * np.median(sample_intervals) * max_s * 0.5

    data = np.empty((len(filtered_pings),max_s))
    data.fill(np.nan)
    index = 0

    for i,pg in enumerate(filtered_pings):
        A=[]
        for p in pg.values():
            try:
                rt = gp.raytracers.RTConstantSVP(p.get_geolocation(),1450)
                sel = pss.apply_selection(p.watercolumn)
                r = p.file_data
                sample_offset = int(round(p.get_geolocation().z / z_resolution))

                samples = p.watercolumn.get_av(sel)
                if linear_mean:
                    samples = np.power(10,0.1*samples)

                index += 1

                m = min(samples.shape[1], max_s-sample_offset)
                a = data[i].copy()

                a[sample_offset:m+sample_offset] = np.nanmean(samples,axis=0)[:m]
                A.append(a)

            except IndexError as e:
                print("error:",i,e,"|",type(e))
            except ValueError as e:
                print("error:",i,e,"|",type(e))
            except RuntimeError as e:
                print("error:",i,e,"|",type(e))

        data[i][:] = np.nanmean(A,axis=0)
        prg.update()

    if linear_mean:
        data = 10*np.log10(data)

    if last_pos is not None:
        last_distance += pnav.navtools.compute_latlon_distance_m(get_latlon(filtered_pings[0]),last_pos)
        
    echograms.append(watercolumn.echograms.EchogramSection(data))
    #echograms[-1].set_pings(filtered_pings)
    echograms[-1].set_ping_numbers(ping_numbers*ping_step)
    echograms[-1].set_ping_times([get_time(p) for p in filtered_pings])
    #echograms[-1].set_ping_distances(pnav.navtools.cumulative_latlon_distances_m([get_latlon(p) for p in filtered_pings]) + last_distance)
    echograms[-1].set_sample_numbers(np.arange(data.shape[1]))
    echograms[-1].set_sample_depths(np.linspace(z_resolution*0.5,max_depth-z_resolution*0.5,data.shape[1]))
    echograms[-1].set_echosounder_depths(echosounder_depths,echosounder_depths_times)
    echograms[-1].set_bottom_depths(bottom_depths,bottom_depths_times)
    echograms[-1].set_ping_parameters("frequency",frequencies)
    
    last_pos = get_latlon(filtered_pings[-1])
    #last_distance = echograms[-1].get_ping_distances()[-1]


In [ ]:
if False:
    import pickle
    path = "/home/data/turbeams/TURBEAMS 2023-12/TURBEAMS-data-crunching/data/W05/em2040/"
    !mkdir -p "$path"

    for i,echogram in enumerate(tqdm(echograms)):
        pickle.dump(echogram,open(path+f'echogram_{i}.pkl','wb'))

In [ ]:
ping_axis = "time"
sample_axis = "depth"

vmin = -85
vmax = -40
interpolation = "nearest"

xlim = np.array([np.nan, np.nan])
ylim = np.array([np.nan, np.nan])

fig = create_figure("test", return_ax = False)
ax= fig.subplots()

E70 = defaultdict(list)
T3 = defaultdict(list)
B3 = defaultdict(list)
T = defaultdict(list)

for ie,e in enumerate(tqdm(echograms[:])):
    # find frequencies
    cf = {frequencies for frequencies in e.get_ping_parameters()['frequency']}
    
    for frequencies in cf:
        ping_numbers = []

        for pi,pf in enumerate(e.get_ping_parameters()['frequency']):
            if pf == frequencies:
                ping_numbers.append(pi)
        
        echo, extent = e.get_echogram(ping_axis=ping_axis, sample_axis=sample_axis, ping_numbers=ping_numbers)
    
        xlim[0] = np.nanmin((xlim[0], extent[0]))
        xlim[1] = np.nanmax((xlim[1], extent[1]))
        ylim[0] = np.nanmax((ylim[0], extent[2]))
        ylim[1] = np.nanmin((ylim[1], extent[3]))
    
        mapable = ax.imshow(
            echo.transpose(),
            extent=extent,
            aspect="auto",
            cmap="YlGnBu_r",
            interpolation=interpolation,
            vmin=vmin,
            vmax=vmax,
        )


        if ping_axis == "time":
            try:
                step = 1
        
                depth_times = (e.get_ping_times_unixtimes()[ping_numbers])[::step]
                depth_times_mdate = unixtime_to_mdate(depth_times)
        
                echosounder_depths = e.echosounder_depth_per_ping_time(depth_times)
                bottom_depths = e.bottom_depth_per_ping_time(depth_times)
                ax.plot(depth_times_mdate, echosounder_depths, color="black")
                #ax.plot(depth_times_mdate, bottom_depths, color="black")
        
                depth_70 = (bottom_depths - echosounder_depths) * 0.7 + echosounder_depths
                depth_90 = (bottom_depths - echosounder_depths) * 0.9 + echosounder_depths
        
                
                
            except Exception as E:
                print('fail NUMBER',ie)
                raise E
                pass
            
    
        #ax.axvline(extent[0], color="grey")
        #ax.axvline(extent[1], color="grey")

ax.set_xlim(xlim)
ax.set_ylim(ylim)

fig.colorbar(mapable, label="dB")

if ping_axis == "time":
    date_format = mdates.DateFormatter("%d-%m-%Y %H:%M:%S")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

#ax.set_xlim(*(valid_times[0]))

In [ ]:
if False:
    !mkdir -p plots
    fig.savefig('plots/colorbar.png')
    fig.savefig('plots/colorbar.svg')

In [ ]:
ping_axis = "time"
sample_axis = "depth"

vmin = -85
vmax = -40
interpolation = "nearest"

xlim = np.array([np.nan, np.nan])
ylim = np.array([np.nan, np.nan])

fig = create_figure("test", return_ax = False)
AX= fig.subplots(sharex=True,nrows=2, height_ratios=[1,1])
ax=AX[0]

E70 = defaultdict(list)
T3 = defaultdict(list)
B3 = defaultdict(list)
T = defaultdict(list)

for ie,e in enumerate(tqdm(echograms[:])):
    # find frequencies
    cf = {frequencies for frequencies in e.get_ping_parameters()['frequency']}
    
    for frequencies in cf:
        ping_numbers = []

        for pi,pf in enumerate(e.get_ping_parameters()['frequency']):
            if pf == frequencies:
                ping_numbers.append(pi)
        
        echo, extent = e.get_echogram(ping_axis=ping_axis, sample_axis=sample_axis, ping_numbers=ping_numbers)
    
        xlim[0] = np.nanmin((xlim[0], extent[0]))
        xlim[1] = np.nanmax((xlim[1], extent[1]))
        ylim[0] = np.nanmax((ylim[0], extent[2]))
        ylim[1] = np.nanmin((ylim[1], extent[3]))
    
        ax.imshow(
            echo.transpose(),
            extent=extent,
            aspect="auto",
            cmap="YlGnBu_r",
            interpolation=interpolation,
            vmin=vmin,
            vmax=vmax,
        )

        if ping_axis == "time":
            try:
                step = 1
        
                depth_times = (e.get_ping_times_unixtimes()[ping_numbers])[::step]
                depth_times_mdate = unixtime_to_mdate(depth_times)
        
                echosounder_depths = e.echosounder_depth_per_ping_time(depth_times)
                bottom_depths = e.bottom_depth_per_ping_time(depth_times)
                ax.plot(depth_times_mdate, echosounder_depths, color="black")
                ax.plot(depth_times_mdate, bottom_depths, color="black")
        
                depth_70 = (bottom_depths - echosounder_depths) * 0.7 + echosounder_depths
                depth_90 = (bottom_depths - echosounder_depths) * 0.9 + echosounder_depths
        
                echo70, extent70 = e.get_echogram_layer(depth_70, depth_90, ping_axis=ping_axis, sample_axis=sample_axis, ping_numbers=ping_numbers)
                # ax.imshow(
                #     echo70.transpose(),
                #     extent=extent70,
                #     aspect="auto",
                #     cmap="jet",
                #     interpolation=interpolation,
                #     vmin=vmin,
                #     vmax=vmax,
                # )
        
                depth_top_3 = echosounder_depths + 3
                depth_top_5 = echosounder_depths + 5
                echot3, extentt3 = e.get_echogram_layer(depth_top_3, depth_top_5, ping_axis=ping_axis, sample_axis=sample_axis, ping_numbers=ping_numbers)
                # ax.imshow(
                #     echot3.transpose(),
                #     extent=extentt3,
                #     aspect="auto",
                #     cmap="Blues",
                #     interpolation=interpolation,
                #     vmin=vmin,
                #     vmax=vmax,
                # )
        
                depth_bottom_3 = bottom_depths - 3
                depth_bottom_5 = bottom_depths - 5
                echob3, extentb3 = e.get_echogram_layer(
                    depth_bottom_5, depth_bottom_3, ping_axis=ping_axis, sample_axis=sample_axis, ping_numbers=ping_numbers
                )
                ax.imshow(
                    echob3.transpose(),
                    extent=extentb3,
                    aspect="auto",
                    cmap="Reds",
                    interpolation=interpolation,
                    vmin=vmin,
                    vmax=vmax,
                )
    
                if False:
                    q = 0.25
                    E70[frequencies].append(10 * np.log10(np.nanquantile(np.power(10, 0.1 * echo70), q=q, axis=1)))
                    T3[frequencies].append(10 * np.log10(np.nanquantile(np.power(10, 0.1 * echot3), q=q, axis=1)))
                    B3[frequencies].append(10 * np.log10(np.nanquantile(np.power(10, 0.1 * echob3), q=q, axis=1)))
                elif False:
                    E70[frequencies].append(10 * np.log10(np.nanmean(np.power(10, 0.1 * echo70), axis=1)))
                    T3[frequencies].append(10 * np.log10(np.nanmean(np.power(10, 0.1 * echot3), axis=1)))
                    B3[frequencies].append(10 * np.log10(np.nanmean(np.power(10, 0.1 * echob3), axis=1)))
                else:
                    E70[frequencies].append(np.nanmean(echo70, axis=1))
                    T3[frequencies].append(np.nanmean(echot3, axis=1))
                    B3[frequencies].append(np.nanmean(echob3, axis=1))
                T[frequencies].append(depth_times_mdate)
                
            except Exception as E:
                print('fail NUMBER',ie)
                raise E
                pass
            
    
        #ax.axvline(extent[0], color="grey")
        #ax.axvline(extent[1], color="grey")

ax.set_xlim(xlim)
ax.set_ylim(ylim)

if ping_axis == "time":
    date_format = mdates.DateFormatter("%d-%m-%Y %H:%M:%S")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

#ax.set_xlim(*(valid_times[0]))

In [ ]:
from matplotlib.pyplot import cm
ax = AX[1]
ax.clear()

features=[
    'e70',
    #'b3',
    #'t3'
]
frequencies = list(E70.keys())
color = cm.rainbow(np.linspace(0, 1, int(len(frequencies)*len(features))))

v = defaultdict(list)

L = [True for _ in frequencies]
for i,freq in enumerate(frequencies):
    for e70,t3,b3,t in zip(E70[freq],T3[freq],B3[freq],T[freq]):

        names = [
            f't3 {freq}',
            f'b3 {freq}',
            f'e70 {freq}',
        ]
        
        if L[i]:
            color_i = i
            if 'e70' in features:
                ax.plot(t,e70,label=f'e70 {freq}', c=color[color_i])
                color_i += 1
            if 'b3' in features:
                ax.plot(t,b3,label=f'b3 {freq}', c=color[color_i])
                color_i += 1
            if 't3' in features:
                ax.plot(t,t3,label=f't3 {freq}', c=color[color_i])
                color_i += 1
            L[i]=False
        else:
            color_i = 0
            if 'e70' in features:
                ax.plot(t,e70,c=color[color_i])
                color_i += 1
            if 'b3' in features:
                ax.plot(t,b3,c=color[color_i])
                color_i += 1
            if 't3' in features:
                ax.plot(t,t3,c=color[color_i])
                color_i += 1

        v[f'{names[0]}'].extend(t3)
        v[f'{names[1]}'].extend(b3)
        v[f'{names[2]}'].extend(e70)

ax.legend()


if ping_axis == 'time':
    date_format = mdates.DateFormatter('%d-%m-%Y %H:%M:%S')
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

for k,V in v.items():
    print(k,np.nanmedian(V))

In [ ]:
fig_wci, ax_wci = create_figure("wci")

In [ ]:
from ipywidgets import *
import ipywidgets as ipw
from time import time

#ping_groups, transducers = group_dual_pings(fm.pings().get_sorted_by_time())

fig_wci.set_tight_layout(True)

last_split_plot = 100

#@widgets.interact
#@debounce(0.1)
def update(w):  
    try:
        if w_protect_stack.value:
            if w['owner'] != w_wci_stack:
                if float(w_text_execution_time.value) > 0.5:
                    w_wci_stack.value = w_wci_stack.value * 0.5 / float(w_text_execution_time.value)
        if w_wci_stack.value > 1:
            w_wci.step = int(w_wci_stack.value/2)
        if w_wci_step.value > 0:
            w_wci.step = w_wci_step.value
    except Exception as e:
        pass
    
    
    w_text_num_total.value = str(int(w_text_num_total.value) +1)
    w_text_num_active.value = str(int(w_text_num_active.value) +1)
    
    t = time()
    global a, last_split_plot, ax_wci, fig_wci, ping1, ping2, wci, wci1
    a = w
    #print(w)
    wci_index = w_wci.value
    wci_stack = w_wci_stack.value
    wci_stack_step = w_wci_stack_step.value
    cmin = w_cmin.value
    cmax = w_cmax.value
    aspect = w_aspect.value
    hsize = w_hsize.value
    heads = w_heads.value
    interpolation = w_interpolation.value
    maxz = w_z.value
    from_bottom = w_from_bottom.value
    threshold_white = w_threshold.value
    linear_mean = w_linear_stack.value
     
    ping_group = ping_groups[wci_index]
    
    wci_time.set_trait('value',ptools.timeconv.unixtime_to_datestring(get_time(ping_group),format='%d-%m-%Y %H:%M:%S'))
    
    if wci_stack > 1:
        max_index = wci_index+wci_stack
        if max_index > len(ping_groups):
            max_index = len(ping_groups)
        pings = []
        for pp in ping_groups[wci_index:max_index:wci_stack_step]:
            for p in pp.values():
                pings.append(p)
                
        #pings = pings[::2]
    

    try:
        if wci_stack > 1:
            wci,extent = mi.make_wci_stack(pings,hsize,progress_bar=progress_bar,linear_mean=linear_mean,from_bottom_xyz=from_bottom, compute_av=w_av.value)
            split_plot=False
        elif len(ping_group) == 1:
            ping = list(ping_group.values())[0]
            split_plot=False
            if heads == 'split_dual_rect':
                if w_av.value:
                    wci = ping.watercolumn.get_av()
                else:
                    wci = ping1.watercolumn.get_amplitudes()
                extent = [0, ping.watercolumn.get_number_of_beams(),0, ping.watercolumn.get_number_of_samples_per_beam()[0]]
            else:
                wci,extent = mi.make_wci(ping,hsize,from_bottom_xyz=from_bottom)
        else:        
            ping1 = ping_group['TRX-2004']
            ping2 = ping_group['TRX-2031']
            if heads == 'blend_dual':
                wci,extent = mi.make_wci_dual_head(ping1,ping2,hsize,from_bottom_xyz=from_bottom, compute_av=w_av.value)
                split_plot=False
            if heads == 'blend_dual_inverw_wci_stackse':
                wci,extent = mi.make_wci_dual_head(ping2,ping1,hsize,from_bottom_xyz=from_bottom, compute_av=w_av.value)
                split_plot=False
            if heads == 'split_dual':
                wci1,extent1 = mi.make_wci(ping1,hsize,from_bottom_xyz=from_bottom, compute_av=w_av.value)
                wci2,extent2 = mi.make_wci(ping2,hsize,from_bottom_xyz=from_bottom, compute_av=w_av.value)
                split_plot=True
            if heads == 'split_dual_rect':
                if w_av.value:
                    wci1 = ping1.watercolumn.get_av()
                    wci2 = ping2.watercolumn.get_av()
                else:
                    wci1 = ping1.watercolumn.get_amplitudes()
                    wci2 = ping2.watercolumn.get_amplitudes()
                extent1 = [0, ping1.watercolumn.get_number_of_beams(),0, ping1.watercolumn.get_number_of_samples_per_beam()[0]]
                extent2 = [0, ping2.watercolumn.get_number_of_beams(),0, ping2.watercolumn.get_number_of_samples_per_beam()[0]]
                split_plot=True
        # if heads == 'both':
        #     wci1,extent1 = make_image(ping1,hsize,from_bottom=from_bottom)
        #     wci2,extent2 = make_image(ping2,hsize,from_bottom=from_bottom)
        if split_plot:
            if last_split_plot != split_plot:
                fig_wci.clear()
                ax_wci = fig_wci.subplots(ncols=2)
                last_split_plot = True
                
            ax_wci[0].clear()
            ax_wci[1].clear()
                
            mapable = ax_wci[0].imshow(wci1.transpose(),aspect=aspect, extent = extent1, cmap='YlGnBu_r',vmin=cmin, vmax=cmax,interpolation=interpolation)
            mapable = ax_wci[1].imshow(wci2.transpose(),aspect=aspect, extent = extent2, cmap='YlGnBu_r',vmin=cmin, vmax=cmax,interpolation=interpolation)
            
            if not heads == 'split_dual_rect':
                if not maxz == -1:
                    ax_wci[0].set_ylim(maxz,0)
                    ax_wci[1].set_ylim(maxz,0)
        else:
            if last_split_plot != split_plot:
                fig_wci.clear()
                ax_wci = fig_wci.subplots(ncols=1)
                last_split_plot = False
            
            ax_wci.clear()
            if threshold_white:
                wci[wci < cmin] = np.nan
                
            mapable = ax_wci.imshow(wci.transpose(),aspect=aspect, extent = extent, cmap='YlGnBu_r',vmin=cmin,vmax=cmax, interpolation=interpolation)
            
            if not maxz == -1:
                ax_wci.set_ylim(maxz,0)
                               
        w_text_num_active.value = str(int(w_text_num_active.value) -1)
        w_text_execution_time.value = str(round(time()-t,3))
            
        
    except Exception as e:
        print(e)
        #pass
        raise (e)


w_z = FloatSlider(min=-1, max=50, step=1, value = -1)
w_cmin = FloatSlider(min=-150, max=150, step=5, value = -90)
w_cmax = FloatSlider(min=-150, max=150, step=5, value = -20)
w_wci = IntSlider(min=0, max=len(ping_groups)-1, step=1, value =0)
w_hsize = IntSlider(min=1, max=2048, step=1, value = 1024)

w_from_bottom = Checkbox(description="from bottom", value=False)
w_linear_stack = Checkbox(description="linear stack", value=True)
w_protect_stack = Checkbox(description="protect stacking time", value=False)
w_threshold = Checkbox(description="threshhold white", value=False)
w_av = Checkbox(description="compute AV", value=True)

w_aspect = Dropdown(options=['auto', 'equal'], value='equal')
w_heads = Dropdown(options=['blend_dual', 'blend_dual_inverse', 'split_dual', 'split_dual_rect'], value='blend_dual')
w_interpolation = Dropdown(options=['antialiased', 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman'], value='nearest')
w_wci_stack = IntText(
    value=1,
    description='stack:',
    disabled=False
)
w_wci_stack_step = IntText(
    value=1,
    description='stack step:',
    disabled=False
)
w_wci_step = IntText(
    value=1,
    description='ping step:',
    disabled=False
)

w_text_num_total = Text(
    value='0',
    placeholder='0',
    description='Total executions:',
    disabled=False   
)
w_text_num_active = Text(
    value='0',
    placeholder='0',
    description='Active executions:',
    disabled=False   
)
w_text_execution_time = Text(
    value='0',
    placeholder='0',
    description='Time of last execution:',
    disabled=False   
)
w_progress = IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Stacking:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
progress_bar = fake_tqdm(w_progress)

wci_time = ipw.Label()

box_ping_slider = HBox([w_wci, wci_time])
box_text = HBox([w_text_num_total,w_text_num_active,w_text_execution_time])
box_options = HBox([w_aspect,w_heads,w_interpolation,w_wci_stack,w_wci_stack_step,w_wci_step])
box_check = HBox([w_from_bottom,w_linear_stack,w_protect_stack, w_threshold, w_av])

w_z.observe(update, names=['value'])
w_cmin.observe(update, names=['value'])
w_cmax.observe(update, names=['value'])
w_wci.observe(update, names=['value'])
w_wci_stack.observe(update, names=['value'])
w_wci_stack_step.observe(update, names=['value'])
w_wci_step.observe(update, names=['value'])
w_hsize.observe(update, names=['value'])
w_aspect.observe(update, names=['value'])
w_from_bottom.observe(update, names=['value'])
w_threshold.observe(update, names=['value'])
w_av.observe(update, names=['value'])
w_linear_stack.observe(update, names=['value'])
w_heads.observe(update, names=['value'])
w_interpolation.observe(update, names=['value'])


update(0)
display(fig_wci.canvas,w_progress, box_text, box_options, box_check, w_z, w_cmin,w_cmax,box_ping_slider,w_hsize)